# Tarea 5
## Fuentes de Datos
## Sebastian Dulong Salazar 
## C.U. 188172

In [63]:
import pandas as pd
import numpy as np

Pregunta 1

In [64]:
covid = pd.read_csv("/mnt/c/Users/ssds6/Documents/fd/casos_nacionales_covid-19.csv") #Datos obtenidos 16-nov-2021

In [65]:
covid.drop(['Unnamed: 0'], axis=1, inplace=True)

In [66]:
covid['fecha_ingreso'] = pd.to_datetime(covid['fecha_ingreso'], infer_datetime_format = '%Y-%M-%D')
covid['fecha_sintomas'] = pd.to_datetime(covid['fecha_sintomas'], infer_datetime_format = '%Y-%M-%D')
covid['fecha_def'] = pd.to_datetime(covid['fecha_def'], infer_datetime_format = '%Y-%M-%D')

Pregunta 2

In [67]:
defunciones = covid.dropna(subset=['fecha_def'])

In [68]:
df = defunciones.entidad_res.value_counts().rename_axis('entidades').reset_index(name='num_defunciones')

In [69]:
df.head(10)

,entidades,num_defunciones
0,MÉXICO,16662
1,HIDALGO,528
2,PUEBLA,298
3,GUERRERO,193
4,MORELOS,179
5,VERACRUZ DE IGNACIO DE LA LLAVE,117
6,OAXACA,89
7,MICHOACÁN DE OCAMPO,81
8,TLAXCALA,72
9,GUANAJUATO,62


Pregunta 3

In [70]:
cmb = defunciones.melt(id_vars='id_registro', value_vars=['neumonia', 'embarazo', 'diabetes', 'epoc', 'asma', 'inmusupr', 'hipertension', 'otra_com', 'cardiovascular', 'obesidad', 'renal_cronica', 'tabaquismo', 'otro_caso'], var_name="comorbilidad", value_name="valor")

In [71]:
cmb2 = cmb[cmb['valor'] == 'SI']
cmb2.comorbilidad.value_counts()

neumonia          50683
hipertension      26990
diabetes          23121
otro_caso         14522
obesidad          13398
tabaquismo         6534
renal_cronica      4521
otra_com           3649
cardiovascular     3576
epoc               2962
inmusupr           2001
asma                873
embarazo             40
Name: comorbilidad, dtype: int64

In [72]:
def2 = defunciones[['id_registro', 'entidad_res']]
cmb_ent = cmb2.merge(def2, how='left', on='id_registro')
cmb_ent.groupby(['comorbilidad','entidad_res']).count()

id_registro  valor
comorbilidad entidad_res                                        
asma         GUANAJUATO                                 1      1
             GUERRERO                                   2      2
             HIDALGO                                    4      4
             JALISCO                                    2      2
             MICHOACÁN DE OCAMPO                        2      2
...                                                   ...    ...
tabaquismo   SINALOA                                    1      1
             TAMAULIPAS                                 3      3
             TLAXCALA                                   8      8
             VERACRUZ DE IGNACIO DE LA LLAVE            7      7
             YUCATÁN                                    1      1

[208 rows x 2 columns]

Pregunta 4

In [73]:
sobrevivientes = covid.drop(defunciones.index)

In [74]:
sobrevivientes.intubado.value_counts(normalize=True)

NO APLICA          0.968762
NO                 0.029411
SI                 0.001674
NO ESPECIFICADO    0.000154
Name: intubado, dtype: float64

In [75]:
cmb_sob = sobrevivientes.melt(id_vars='id_registro', value_vars=['neumonia', 'embarazo', 'diabetes', 'epoc', 'asma', 'inmusupr', 'hipertension', 'otra_com', 'cardiovascular', 'obesidad', 'renal_cronica', 'tabaquismo', 'otro_caso'], var_name="comorbilidad", value_name="valor")
cmb2_sob = cmb_sob[cmb_sob['valor'] == 'SI']
sob2 = sobrevivientes[['id_registro', 'entidad_res']]
cmb_ent_sob = cmb2_sob.merge(sob2, how='left', on='id_registro')
cmb_ent_sob.groupby(['comorbilidad','entidad_res']).count()

id_registro  valor
comorbilidad entidad_res                                        
asma         DURANGO                                    6      6
             GUANAJUATO                                21     21
             GUERRERO                                  31     31
             HIDALGO                                  128    128
             JALISCO                                   19     19
...                                                   ...    ...
tabaquismo   TAMAULIPAS                                31     31
             TLAXCALA                                 123    123
             VERACRUZ DE IGNACIO DE LA LLAVE          179    179
             YUCATÁN                                   26     26
             ZACATECAS                                 10     10

[288 rows x 2 columns]

Pregunta 5

In [102]:
defunciones_hosp = defunciones[defunciones['tipo_paciente'] == 'HOSPITALIZADO']
defunciones_hosp.dropna(subset=['fecha_def'])
defunciones_hosp.dropna(subset=['fecha_ingreso'])
defunciones_hosp['dias_hospitalizado'] = (defunciones_hosp['fecha_def'] - defunciones_hosp['fecha_ingreso']).dt.days

/tmp/ipykernel_887/1949668056.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  defunciones_hosp['dias_hospitalizado'] = (defunciones_hosp['fecha_def'] - defunciones_hosp['fecha_ingreso']).dt.days


In [103]:
defunciones_hosp['dias_hospitalizado'].describe()

count    65046.000000
mean         9.014390
std         11.576827
min       -365.000000
25%          3.000000
50%          6.000000
75%         12.000000
max        430.000000
Name: dias_hospitalizado, dtype: float64

In [104]:
defunciones_hosp = defunciones_hosp.drop(defunciones_hosp.index[defunciones_hosp['dias_hospitalizado'] < 0])

In [105]:
defunciones_hosp['dias_hospitalizado'].describe()

count    65001.000000
mean         9.031584
std         11.477234
min          0.000000
25%          3.000000
50%          6.000000
75%         12.000000
max        430.000000
Name: dias_hospitalizado, dtype: float64

In [106]:
defunciones_hosp = defunciones_hosp.drop(defunciones_hosp.index[defunciones_hosp['dias_hospitalizado'] <= 0])

In [107]:
defunciones_hosp['dias_hospitalizado'].describe()

count    59673.000000
mean         9.837984
std         11.642812
min          1.000000
25%          3.000000
50%          7.000000
75%         13.000000
max        430.000000
Name: dias_hospitalizado, dtype: float64

En total, 65046 personas hospitalizadas murieron.
En promedio, morían cada 9 días, con una desviación estándar de 11.5 días (los datos mayores a 9 días eran relativamente pocos y mucho mayores 9 días para que con esa desviación estándar se llegara a ese promedio)
El mínimo es -365 por un registro cuya fecha de defunción está registrada 1 año antes que la de ingreso (error humano). Al quitar ese, hay 2 que tienen -31 y luego, algunos con -27. Hay muchos errores de registro. Al quitar todos los menores a 0 (que no tienen sentido) nos queda que el mínimo es 0 días de hospitalización (salieron el mismo día), y sin esos, el mínimo es 1 día.
El 50% de los datos se encuentra antes de un número menor al promedio, y después de ahí hay menor cantidad de datos por cuartil - están más separados
El que más duró hospitalizado estuvo por más de un año (430 días)